In [1]:
import tensorflow as tf
from tensorflow import keras

In [6]:
!pip install -q -U keras-tuner


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
import keras_tuner as kt

In [8]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [9]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [11]:
#zdefiniowanie modelu
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [13]:
#utowrzenie wystąpienie tunera i hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
#wyszukiwanie hiperparametrów
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Wyszukiwanie hiperparametrów zakończone.
Optymalna liczba neuronów warrstwy ukrytej: {best_hps.get('units')}.{best_hps.get('learning_rate')}.
          optymalna wartość learning_rate: 
          
""")

Trial 7 Complete [00h 00m 11s]
val_accuracy: 0.8634999990463257

Best val_accuracy So Far: 0.8727499842643738
Total elapsed time: 00h 01m 43s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
480               |512               |units
0.0001            |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
1490/1500 [============================>.] - ETA: 0s - loss: 0.6247 - accuracy: 0.7956